In [1]:
import json

with open("../recorded_sequences/seq_0.json") as f:
    data = json.load(f)

print(type(data))
print(len(data))
print(data[0].keys())


<class 'list'>
30
dict_keys(['hands', 'face'])


In [2]:
def flatten_frame(frame):
    features = []

    for hand in frame.get("hands", []):
        for x, y, z in hand:
            features.extend([x, y, z])

    return features


flat = flatten_frame(data[0])
print(len(flat))


0


In [3]:
import numpy as np

sequence = [flatten_frame(frame) for frame in data]
X = np.array(sequence)

print(X.shape)


(30, 0)


In [4]:
print(data[0])


{'hands': [], 'face': [[[0.5579839944839478, 0.6166870594024658, -0.02875036932528019], [0.5604438185691833, 0.5709068179130554, -0.056257519870996475], [0.5565325617790222, 0.5853782892227173, -0.028836369514465332], [0.5463018417358398, 0.5290401577949524, -0.04597121477127075], [0.5603307485580444, 0.557289183139801, -0.060338810086250305], [0.5581648945808411, 0.5401163101196289, -0.05674196034669876], [0.5510527491569519, 0.4991340935230255, -0.03022964671254158], [0.46526506543159485, 0.5028902888298035, -0.004769605118781328], [0.5484620332717896, 0.46780872344970703, -0.024902502074837685], [0.5481870174407959, 0.4494101405143738, -0.02790159545838833], [0.5434510111808777, 0.3791871666908264, -0.019102802500128746], [0.5580167770385742, 0.6233956217765808, -0.027118545025587082], [0.5573236346244812, 0.6286390423774719, -0.023316921666264534], [0.5561286807060242, 0.6308543682098389, -0.01843336783349514], [0.5569299459457397, 0.6358347535133362, -0.015989284962415695], [0.557

In [5]:
import numpy as np

X = np.array([flatten_frame(frame) for frame in data])
X = X.astype("float32")

print("timesteps:", X.shape[0])
print("features:", X.shape[1])


timesteps: 30
features: 0


In [6]:
import torch
import torch.nn as nn

timesteps, feature_dim = X.shape

class GestureRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=128):
        super().__init__()
        self.rnn = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)  # one gesture (demo)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = out[:, -1, :]      # last timestep
        out = self.fc(out)
        return out

model = GestureRNN(feature_dim)
print(model)


GestureRNN(
  (rnn): LSTM(0, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
)


In [7]:
dummy = torch.tensor(X).unsqueeze(0)  # (1, timesteps, features)
out = model(dummy)
print(out.shape)


torch.Size([1, 1])


In [8]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

y = torch.tensor([[1.0]])  # dummy label (one gesture)

model.train()
optimizer.zero_grad()
output = model(dummy)
loss = criterion(output, y)
loss.backward()
optimizer.step()

print("loss:", loss.item())


loss: 0.9861351251602173
